In [0]:
%run "../includes/configuration"

In [0]:
from pyspark.sql.functions import sum, desc, when, count,col,rank
from pyspark.sql.window import Window

In [0]:
race_result_df = spark.read.parquet(f"{presentation_folder_path}/races_result")

In [0]:
driver_standing_df = race_result_df\
                    .groupBy('race_year', 'driver_name','driver_nationality', 'team')\
                    .agg(sum('points').alias('total_points'),
                         count(when(col('position') == 1, True)).alias('wins'))\
                    .orderBy(desc('total_points'))

In [0]:
driver_rank_spec = Window.partitionBy('race_year').orderBy(desc('total_points'), desc('wins'))
final_driver_standing_df = driver_standing_df.withColumn('driver_rank', rank().over(driver_rank_spec))

In [0]:
display(final_driver_standing_df)

In [0]:
#final_driver_standing_df.write.mode('overwrite').parquet(f"{presentation_folder_path}/driver_standing")

In [0]:
final_driver_standing_df.write.mode('overwrite').format("parquet").saveAsTable("f1_presentation.driver_standing")

In [0]:
%sql
SELECT * FROM f1_presentation.driver_standing